In [1]:
import pandas as pd
import numpy as np
import time
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC

In [2]:
seasons = pd.read_csv("Data/RegularSeasonData/pbp2009.csv")
for x in range(7):
    data = pd.read_csv("Data/RegularSeasonData/pbp201"+str(x)+".csv")
    seasons = seasons.append(data)

C:\Users\skrzym\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2902: DtypeWarning: Columns (46) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\skrzym\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2902: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\skrzym\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2902: DtypeWarning: Columns (26,46) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
#cdata = seasons[['qtr', 'down', 'time', 'TimeUnder', 'yrdline100', 'ScoreDiff', 'PlayType']]
cdata = seasons[['qtr', 'down', 'TimeUnder', 'yrdline100', 'ScoreDiff', 'PlayType']]

In [4]:
valid_play_types = [
    'Field Goal',
    'Pass',
    'Run',
    'QB Kneel',
    'Punt'
]
def match(playtype):
    return playtype in valid_play_types

In [5]:
cdata = cdata[cdata.PlayType.map(match)]
cdata = cdata[cdata['down'].isnull()==False]

#### Put each vector into a list and create a list of lists shaped like x and put playtype in y for classification training

In [14]:
X = [list(rec) for rec in cdata[['qtr', 'down', 'TimeUnder', 'yrdline100', 'ScoreDiff']].to_records(index=False)]
Y = cdata['PlayType'].tolist()

X = np.array(X)
Y = np.array(Y)

In [8]:
def build_bagging_model(x,y):
    clf = OneVsRestClassifier(BaggingClassifier(SVC(kernel='linear', probability=True, class_weight='balanced'), max_samples=1.0 / 1000, n_estimators=10))
    clf.fit(x,y)
    return clf
    print(clf.predict_proba(x))

def build_random_forest_model(x,y):
    clf = RandomForestClassifier(min_samples_leaf=20)
    clf.fit(x,y)
    return clf
    print(clf.score(x,y))
    print(clf.predict_proba(x[5]))

In [12]:
start = time.time()
rfc = build_random_forest_model(X,Y) # less than 5 seconds and uses full dataset
stop = time.time()
print(stop - start)
rfc.score(X,Y)

2.7429697513580322


0.71186992801222826

In [15]:
start = time.time()
bc = build_bagging_model(X,Y) # takes about 3-4 minutes on my machine and it's sampling very little of the data
stop = time.time()
print(stop - start)
bc.score(X,Y)

8.418323993682861


0.59648545030839117

In [16]:
qtr = 4
down = 4
TimeUnder = 1
yrdline100 = 20
ScoreDiff = -3

test_case = [[qtr,down,TimeUnder,yrdline100,ScoreDiff]]
classes = rfc.classes_
bcp = bc.predict_proba(test_case)[0]*100
rfcp = rfc.predict_proba(test_case)[0]*100

bcp = [str(round(x,2)) for x in bcp]
rfcp = [str(round(x,2)) for x in rfcp]

print("Bagging SVC")
for item in zip(classes,bcp):
    print(item)

print("")
print("Random Forest")
for item in zip(classes,rfcp):
    print(item)



Bagging SVC
('Field Goal', '28.07')
('Pass', '41.02')
('Punt', '18.77')
('QB Kneel', '2.71')
('Run', '9.44')

Random Forest
('Field Goal', '85.19')
('Pass', '11.17')
('Punt', '0.76')
('QB Kneel', '0.0')
('Run', '2.88')
